In [1]:
# Imports

import numpy as np
import numpy.matlib
import pandas as pd
import matplotlib.pyplot as plt

In [47]:
# Get the excel file and read

class ExcelReader(object):
    
    def __init__(self, path, ecID=1, vals=None):
        self.path = path
        self.ecID = ecID
        
        self.vals = vals
        
        # Placeholder dictionary to place the next ones
        self.parameterData = {}
        self.pD_network = {}
        
        # Resources
        self.resources = {}
        
        # General Info dictionary
        self.generalInfo = {}
        
        # Branch dictionary
        self.puSist = {}
        self.network = {}
        
        self.read_generalInfo()
        self.read_brachData()
        self.read_genData()
        
        return
    
    
    def read_generalInfo(self):
        # General information sheet from excel
        generalInfo = pd.read_excel(self.path, sheet_name='General_Information', header=None)
        self.resources['numEC'] = generalInfo.values[2, 3]
        self.resources['period'] = generalInfo.values[3, 3]
        self.resources['periodDuration'] = generalInfo.values[4, 3]
        self.resources['owners'] = generalInfo.values[5, 3]
        
        # Read max imports and export quantities
        self.generalInfo['P_Max_Imp'] = generalInfo.values[23, 5:]
        self.generalInfo['P_Max_Exp'] = generalInfo.values[24, 5:]
        self.generalInfo['Energy_Buy_Price'] = generalInfo.values[25, 5:]
        self.generalInfo['Energy_SellPrice'] = generalInfo.values[26, 5:]
        self.generalInfo['GEE_CofA'] = generalInfo.values[27, 5:]
        self.generalInfo['GEE_CofB'] = generalInfo.values[28, 5:]
        self.generalInfo['GEE_CofC'] = generalInfo.values[29, 5:]
        
        return
    
    
    def read_brachData(self):
        # Branch data sheet from excel
        sheet_name_branch = 'Network_Info'
        data_branch = pd.read_excel(self.path, sheet_name=sheet_name_branch, header=None)
        
        # Set the values
        self.puSist['basePower'] = data_branch.values[1, 8]
        self.puSist['baseVoltage'] = data_branch.values[2, 8]
        self.puSist['baseCurrent'] = data_branch.values[3, 8]
        self.puSist['baseImpendance'] = data_branch.values[4, 8]
        
        busRef = data_branch.values[2, :3]
        busAux = np.unique(data_branch.values[22:, 1:3])
        
        voltageMax = np.zeros((max(busAux), self.resources['period']))
        voltageMin = np.zeros((max(busAux), self.resources['period']))
        angleMax = np.zeros((max(busAux), self.resources['period']))
        angleMin = np.zeros((max(busAux), self.resources['period']))
        
        for i in busAux:
            voltageMax[i-1, :] = data_branch.values[2, 5]
            voltageMin[i-1, :] = data_branch.values[2, 4]
            angleMax[i-1, :] = data_branch.values[4, 5]
            angleMin[i-1, :] = data_branch.values[4, 4]
        
        voltageMax[busRef[0]-1, :] = busRef[1]
        voltageMin[busRef[0]-1, :] = busRef[1]
        angleMax[busRef[0]-1, :] = busRef[2]
        angleMin[busRef[0]-1, :] = busRef[2]
        
        self.network['voltageMax'] = voltageMax
        self.network['voltageMin'] = voltageMin
        self.network['angleMax'] = angleMax
        self.network['angleMin'] = angleMin
        self.network['busNum'] = np.reshape(busAux, (1, busAux.shape[0]))
        self.network['busRef'] = busRef[0]
        
        # Conversion from data_branch
        numBranch = data_branch.values[:, 0].shape[0] - 22
        ybus = np.zeros((max(busAux), max(busAux)))
        diag = np.zeros((max(busAux), max(busAux)))
        branchID = np.zeros((max(busAux), max(busAux)))
        
        busI = data_branch.values[22:, 1]
        busJ = data_branch.values[22:, 2]
        
        # Admitance set
        for i in np.arange(busI.shape[0]):
            ybus[busI[i]-1, busJ[i]-1] = 1
            ybus[busJ[i]-1, busI[i]-1] = 1
            
            branchID[busI[i]-1, busJ[i]-1] = 1
            
            
        # Diagonal matrix values
        for i in np.arange(ybus.shape[0]):
            ybus[i, i] = 1
            diag[i, i] = 1
            
        branch = np.zeros((max(busAux), max(busAux), 4))
        idBusI = np.matlib.repmat(busI, 1, 4)
        idBusI = np.transpose(idBusI)
        
        idBusJ = np.matlib.repmat(busJ, 1, 4)
        idBusJ = np.transpose(idBusJ)
        
        # Branch and branchInfo
        mask = np.zeros(branch.shape)
        idBranchInfo = np.matlib.repmat(np.arange(1, 5), numBranch, 1)
        idBranchInfo = np.reshape(idBranchInfo.transpose(), (numBranch*4, 1))
        for i in np.arange(idBranchInfo.shape[0]):
            mask[idBusI[i, 0]-1, idBusJ[i, 0]-1, idBranchInfo[i, 0]-1] = 1
            
        branch[mask.astype(bool)] = data_branch.values[22:, 5:9].ravel()
        
        # Line data selection
        lineData = data_branch.values[1:, 0:9]
        lineData = pd.DataFrame(lineData)
        lineData = lineData.loc[(lineData.index != 17) & (lineData.index != 18)]
        for col in lineData.columns:
            lineData[col] = pd.to_numeric(lineData[col], errors='coerce')

        lineData.columns = ['Col{:02d}'.format(i) for i in np.arange(lineData.shape[1])]
        lineData = lineData[['Col00', 'Col01', 'Col02', 'Col05', 'Col06', 'Col07']]
        
        #Power Limit
        powerLimit = data_branch.values[1:, 0:9]
        powerLimit = pd.DataFrame(powerLimit)
        powerLimit = powerLimit.loc[(powerLimit.index != 17) & (powerLimit.index != 18)]
        for col in powerLimit.columns:
            powerLimit[col] = pd.to_numeric(powerLimit[col], errors='coerce')
            
        powerLimit.columns = ['Col{:02d}'.format(i) for i in np.arange(powerLimit.shape[1])]
        powerLimit = powerLimit[['Col00', 'Col01', 'Col02', 'Col08']]
        
        self.network['branch'] = branch
        self.network['lineData'] = lineData.values
        self.network['lineLimit'] = powerLimit
        self.pD_network['ybus'] = ybus
        self.pD_network['diag'] = diag
        self.pD_network['branch'] = branchID
        self.pD_network['numLine'] = numBranch
        return
        
    # TODO: continue from 133
    
    def read_genData(self):
        # Read the Excel
        sheet_name_gen = 'Generator_EC{}'.format(self.ecID)
        data_gen = pd.read_excel(self.path, sheet_name=sheet_name_gen, header=None)
        
        # Conversion from genData
        genOpt, genOptName, idx = self._genOption()
        idSimPer = np.where(data_gen.values[0, :] == self.resources['period'])[0][0]
        data_gen.drop(data_gen.columns[idSimPer+1:], axis=1, inplace=True)
        
        
        #for i in np.arange(len(data_gen.iloc[0])):
        #    #print(data_gen.iloc[0][i])
        #    print(isinstance(data_gen.iloc[0][i], float))
            
        print(isinstance(data_gen.iloc[0], float))
        
        return
    
    
    def _genOption(self):
        if self.vals == None:
            options = [1, 2, 3, 4, 5, 6, 7,
                       1, 2, 3, 4, 5, 6, 7]
        else:
            options = self.val
        
        names = ['photovoltaic', 'wind', 'co-generation', 'biomass', 'waste-to-energy', 'small hydro',
                 'fuel cell', 'external supplier', 'none',
                 'Xup1', 'Xup2', 'Active Min', 'Active Max', 'Reactive Min', 'Reactive Max', 'Excess Max']
        
        idx = {}
        idx['PV'] = options[0]
        idx['Wind'] = options[1]
        idx['CHP'] = options[2]
        idx['Bio'] = options[3]
        idx['WtE'] = options[4]
        idx['SH'] = options[5]
        idx['FC'] = options[6]
        idx['ES'] = options[7]
        idx['No'] = options[8]
        
        return options, names, idx
        
    
    def readExcel(self):

        # Excel sheets to read from the excel
        sheet_name_branch = 'Network_Info'
        sheet_name_peers = 'Peers_Info_EC{}'.format(self.ecID)
        sheet_name_gen = 'Generator_EC{}'.format(self.ecID)
        sheet_name_load = 'Load_EC{}'.format(self.ecID)
        sheet_name_stor = 'Storage_EC{}'.format(self.ecID)
        sheet_name_cstation = 'CStation_EC{}'.format(self.ecID)
        sheet_name_v2g = 'Vehicle_EC{}'.format(self.ecID)

        data_peers = pd.read_excel(path, sheet_name=sheet_name_peers, header=None)
        data_gen = pd.read_excel(path, sheet_name=sheet_name_gen, header=None)
        data_load = pd.read_excel(path, sheet_name=sheet_name_load, header=None)
        data_storage = pd.read_excel(path, sheet_name=sheet_name_stor, header=None)
        data_cstation = pd.read_excel(path, sheet_name=sheet_name_cstation, header=None)
        data_v2g = pd.read_excel(path, sheet_name=sheet_name_v2g, header=None)
        
        return generalInfo

data = ExcelReader(path='src/EC_OnlyMembers_V8/EC_V4.xlsx', ecID=1)

False


In [48]:
data.network['busRef']

1

In [2]:
import os
import datetime 
import pickle
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.utils import shuffle


def get_csv_from_folder(path):
    return  [pd.read_csv(path + file) for file in os.listdir(path)]

def convert_to_timestamped_data(df):
    df['date'] = pd.to_datetime(df['date'])
    df.set_index('date', inplace=True)

def get_indexes_from_net_df_name(df, names):
    indexes_fast = [[int(df[df['name'] == df_name].index[0]) for df_name in df['name'].values if name in df_name] for name in names]
    return [item[0] for item in indexes_fast]

def match_index_with_name(net, df, element_type='bus'):
    mapping = {column: net[element_type].iloc[column]['name'] for column in df.columns.values}
    return df.rename(columns=mapping)

def serialize_object(name, data, message=None):
    '''
    Summary:
    :param name: Name of the output file_path.
    :type name: string
    :param data: Target data to be serialized.
    :type data: Variable.
    :param message: Message to be showed in the console.
    :type message: string
    :return: output PICKLE file_path.
    :rtype: PICKLE file_path
    '''
    start_time = datetime.datetime.now()
    if message:
        print(message + '... Please wait')
    pickle_out = open('{}.pickle'.format(name), 'wb')
    pickle.dump(data, pickle_out)
    pickle_out.close()

    end_time = datetime.datetime.now()
    time_diff = (end_time - start_time)
    execution_time = time_diff.total_seconds()
    if message:
        print(message + ':🗸 (Execution time: {}[s])'.format(execution_time))

def deserialize_object(name, message=None):
    '''
    Summary: Function that takes the name of a PICKLE file_path and deserializes the data into to the python progm.
    :param file: PICKLE input file_path name.
    :type file: string
    :param message: Variable message.
    :type message: string
    :return: Desired object.
    :rtype: object
    '''
    start_time = datetime.datetime.now()
    if message:
        print('Deserializing '  + name + '... Please wait')
    pickle_in = open('{}.pickle'.format(name), 'rb')
    end_time = datetime.datetime.now()
    time_diff = (end_time - start_time)
    execution_time = time_diff.total_seconds()
    result = pickle.load(pickle_in)
    if message:
        print(message + '🗸 (Execution time: {}[s])'.format(execution_time))
    return result

# Scaling reference here https://www.atoti.io/articles/when-to-perform-a-feature-scaling/
# def split_and_suffle(X, y, test_size=0.2, scaling=False):
#     le = LabelEncoder()
#     X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size, shuffle=False)
#     if 'season' in X_train.columns:
#         X_train['season'] = le.fit_transform(X_train['season'])
#         X_test['season'] = le.fit_transform(X_test['season'])   
#     X_train, y_train = shuffle(X_train, y_train)
#     if scaling:
#         from sklearn.preprocessing import MaxAbsScaler
#         scaler = {
#             'X_train': MaxAbsScaler(),
#             'X_test': MaxAbsScaler(),
#             'y_train': MaxAbsScaler(),
#             'y_test': MaxAbsScaler()
#         }
#         X_train = scaler['X_train'].fit_transform(X_train)
#         X_test = scaler['X_test'].fit_transform(X_test)
#         y_train = scaler['y_train'].fit_transform(y_train)
#         y_test = scaler['y_test'].fit_transform(y_test)
#     X_train = pd.DataFrame(X_train, columns=X.columns)
#     X_test = pd.DataFrame(X_test, columns=X.columns)                          
#     y_train = pd.DataFrame(y_train, columns=y.columns)
#     y_test = pd.DataFrame(y_test, columns=y.columns)
#     if scaling:
#         return X_train, X_test, y_train, y_test, scaler                      
#     else: 
#         return X_train, X_test, y_train, y_test
def split_and_suffle(X, y, test_size=0.2, scaling=False):
    le = LabelEncoder()
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size, shuffle=False)
    if 'season' in X_train.columns:
        X_train['season'] = le.fit_transform(X_train['season'])
        X_test['season'] = le.fit_transform(X_test['season'])   
    if scaling:
        from sklearn.preprocessing import MaxAbsScaler
        scaler = {
            'X': MaxAbsScaler(),
            'y': y.max().max()
            } 
        X_train = scaler['X'].fit_transform(X_train)
        X_test = scaler['X'].fit_transform(X_test)
        y_train = y_train / scaler['y']
        y_test = y_test / scaler['y']
    X_train, y_train = shuffle(X_train, y_train)
    X_train = pd.DataFrame(X_train, columns=X.columns).reset_index(drop=True)
    X_test = pd.DataFrame(X_test, columns=X.columns).reset_index(drop=True)
    y_train = pd.DataFrame(y_train, columns=y.columns).reset_index(drop=True)
    y_test = pd.DataFrame(y_test, columns=y.columns).reset_index(drop=True)
    if scaling:
        return X_train, X_test, y_train, y_test, scaler                      
    else: 
        return X_train, X_test, y_train, y_test
def unscale_df(df, scaler):
    return pd.DataFrame(scaler.inverse_transform(df), columns=df.columns)
def convert_df_to_bool(df):
    for col in df.columns:
        df[col] = df[col].astype(bool)
    return df

cols_with_positive_values = lambda df: [col for col in df.columns if df[col].any()]
# threshold related
def compute_threshold(df):
    X_train, X_test, y_train, y_test = split_and_suffle(pd.DataFrame(np.ones((df.shape[0],10))), df)
    return min(0.0025, y_test.loc[:, y_test.max(axis=0) != 0].max(axis=0).mean() * 0.10)
def check_positive_count(reg_data, class_data, threshold, experiment='max_u'):
    print('Positive count in classification data', experiment, ':', count_positives_class(class_data))
    print('Positive count in regression data', experiment, 'with threshold', threshold, ':', count_positives_reg(reg_data, threshold))
    print('\n')
def check_negative_count(reg_data, class_data, threshold, experiment='max_u'):
    print('Negative count in classification data', experiment, ':', class_data.shape[0] * class_data.shape[1] - class_data.sum().sum())
    print('Negative count in regression data', experiment, 'with threshold', threshold, ':', count_negatives_reg(reg_data, threshold))
    print('\n')
def count_positives_class(class_data):
    return class_data.sum().sum()
def count_negatives_class(class_data):
    return class_data.shape[0] * class_data.shape[1] - class_data.sum().sum()
def count_positives_reg(reg_data, threshold):
    return reg_data[reg_data > threshold].count().sum()
def count_negatives_reg(reg_data, threshold):
    return reg_data[reg_data < threshold].count().sum()

In [ ]:
#%%
import os
import pandas as pd
import pandapower as pp
from pandapower.plotting.plotly import simple_plotly
from pandapower.plotting.plotly import pf_res_plotly
import numpy as np
from abc import ABC, abstractmethod
f#rom thesis_package import extractor as ex, utils as ut
##############################################################################
############################## Element Types #################################
##############################################################################
# Element classses from the excel file
class Element(ABC):
    def __init__(self, id, internal_bus_location, manager, owner, type_of_contract):
        self.id = id
        self.internal_bus_location = internal_bus_location
        self.manager = manager
        self.owner = owner
        self.type_of_contract = type_of_contract
    @abstractmethod
    def set_profile(self):
        pass
# class Load extends Element class and adds the values of: load, charge type, power contracted, power factor.
class Load(Element):
    def __init__(self, id, internal_bus_location, manager, owner, type_of_contract, charge_type, power_contracted_kw, tg_phi):
        super().__init__(id, internal_bus_location, manager, owner, type_of_contract)
        self.charge_type = charge_type
        self.power_contracted_kw = power_contracted_kw
        self.tg_phi = tg_phi        
    def set_profile(self,
                    p_real_kw=None,
                    q_real_kvar=None,
                    p_reduce_kw=None,
                    p_cut_kw=None,
                    p_move_kw=None,
                    p_in_move_kw=None,
                    cost_reduce_mu=None,
                    cost_cut_mu=None,
                    cost_mov_mu=None,
                    cost_ens_mu=None):
        self.p_real_kw = p_real_kw
        self.q_real_kvar = q_real_kvar
        self.p_reduce_kw = p_reduce_kw
        self.p_cut_kw = p_cut_kw
        self.p_move_kw = p_move_kw
        self.p_in_move_kw = p_in_move_kw
        self.cost_reduce_mu = cost_reduce_mu
        self.cost_cut_mu = cost_cut_mu
        self.cost_mov_mu = cost_mov_mu
        self.cost_ens_mu = cost_ens_mu
# class Generator extends Element class and adds the values of: type_of_generator, p_max_kw, q_max_kw, q_min_kw.
class Generator(Element):
    def __init__(self, id, internal_bus_location, manager, owner, type_of_contract, type_of_generator, p_max_kw, p_min_kw, q_max_kvar, q_min_kvar):
        super().__init__(id, internal_bus_location, manager, owner, type_of_contract)
        self.type_of_generator = type_of_generator
        self.p_max_kw = p_max_kw
        self.p_min_kw = p_min_kw
        self.q_max_kvar = q_max_kvar
        self.q_min_kvar = q_min_kvar
    def set_profile(self, power_real_kw=None, cost_parameter_a_mu=None, cost_parameter_b_mu=None, cost_parameter_c_mu=None, cost_nde_mu=None, ghg_cof_a_mu=None, ghg_cof_b_mu=None, ghg_cof_c_mu=None):
        self.p_real_kw = power_real_kw
        self.cost_parameter_a_mu = cost_parameter_a_mu
        self.cost_parameter_b_mu = cost_parameter_b_mu
        self.cost_parameter_c_mu = cost_parameter_c_mu
        self.cost_nde_mu = cost_nde_mu
        self.ghg_cof_a_mu = ghg_cof_a_mu
        self.ghg_cof_b_mu = ghg_cof_b_mu
        self.ghg_cof_c_mu = ghg_cof_c_mu    
# class Storage extends Element class and adds the values of: battery_type, energy_capacity_kVAh, energy_min_percent, charge_efficiency_percent, discharge_efficiency_percent, initial_state_percent, p_charge_max_kw, p_discharge_max_kw.
class Storage(Element):
    def __init__(self, id, internal_bus_location, manager, owner, type_of_contract, battery_type, energy_capacity_kvah, energy_min_percent, charge_efficiency_percent, discharge_efficiency_percent, initial_state_percent, p_charge_max_kw, p_discharge_max_kw):
        super().__init__(id, internal_bus_location, manager, owner, type_of_contract)
        self.battery_type = battery_type
        self.energy_capacity_kvah = energy_capacity_kvah
        self.energy_min_percent = energy_min_percent
        self.charge_efficiency_percent = charge_efficiency_percent
        self.discharge_efficiency_percent = discharge_efficiency_percent
        self.initial_state_percent = initial_state_percent
        self.p_charge_max_kw = p_charge_max_kw
        self.p_discharge_max_kw = p_discharge_max_kw
    def set_profile(self, power_charge_limit_kw=None, power_discharge_limit_kw=None, charge_price_mu=None, discharge_price_mu=None):
        self.p_charge_limit_kw = power_charge_limit_kw
        self.p_discharge_limit_kw = power_discharge_limit_kw
        self.charge_price_mu = charge_price_mu
        self.discharge_price_mu = discharge_price_mu
# class Chargring_Station extends Element class and adds the values of:, p_charge_max_kw, p_discharge_max_kw, charge_efficiency_percent, discharge_efficienc_percent, energy_capacity_max_kwh, place_start, place_end.
class Charging_Station(Element):
    def __init__(self, id, internal_bus_location, manager, owner, type_of_contract, p_charge_max_kw, p_discharge_max_kw, charge_efficiency_percent, discharge_efficiency_percent, energy_capacity_max_kwh, place_start, place_end):
        super().__init__(id, internal_bus_location, manager, owner, type_of_contract)
        self.p_charge_max_kw = p_charge_max_kw
        self.p_discharge_max_kw = p_discharge_max_kw
        self.charge_efficiency_percent = charge_efficiency_percent
        self.discharge_efficiency_percent = discharge_efficiency_percent
        self.energy_capacity_max_kwh = energy_capacity_max_kwh
        self.place_start = place_start
        self.place_end = place_end
    # Override the method set_profile, setting the profiles of power_charge_limit_kw and p_discharge_limit_kw.
    def set_profile(self, power_charge_limit_kw=None, p_discharge_limit_kw=None):
        self.p_charge_limit_kw = power_charge_limit_kw
        self.p_discharge_limit_kw = p_discharge_limit_kw
##############################################################################
############################## Other Elements ################################
##############################################################################
class Vehicle:
    def __init__(self, id, manager, owner, type_of_contract, type_of_vehicle, energy_capacity_max_kwh, p_charge_max_kw, p_discharge_max_kw, charge_efficiency_percent, discharge_efficiency_percent, initial_state_SOC_percent, minimun_technical_SOC_percent):
        self.id = id
        self.manager = manager
        self.owner = owner
        self.type_of_contract = type_of_contract
        self.type_of_vehicle = type_of_vehicle
        self.energy_capacity_max_kwh = energy_capacity_max_kwh
        self.p_charge_max_kw = p_charge_max_kw
        self.p_discharge_max_kw = p_discharge_max_kw
        self.charge_efficiency_percent = charge_efficiency_percent
        self.discharge_efficiency_percent = discharge_efficiency_percent
        self.initial_state_SOC_percent = initial_state_SOC_percent
        self.minimun_technical_SOC_percent = minimun_technical_SOC_percent
    def set_profile(
        self, arrive_time_period=None, departure_time_period=None,
        place=None, used_soc_percent_arriving=None, soc_percent_arriving=None,
        soc_required_percent_exit=None, p_charge_max_constracted_kw=None,
        p_discharge_max_constracted_kw=None, charge_price=None, discharge_price=None):
        self.arrive_time_period = arrive_time_period
        self.departure_time_period = departure_time_period
        self.place = place
        self.used_soc_percent_arriving = used_soc_percent_arriving
        self.soc_percent_arriving = soc_percent_arriving
        self.soc_required_percent_exit = soc_required_percent_exit
        self.p_charge_max_constracted_kw = p_charge_max_constracted_kw
        self.p_discharge_max_constracted_kw = p_discharge_max_constracted_kw
        self.charge_price = charge_price
        self.discharge_price = discharge_price
# class peers, which containts the properties: type_of_peer, type_of_contract, owner.
class Peer:
    def __init__(self, id, type_of_peer, type_of_contract):
        self.id = id
        self.type_of_peer = type_of_peer
        self.type_of_contract = type_of_contract
    # Property method that sets the buy_price_mu and sell_price_mu
    def set_profile(self, buy_price_mu=None, sell_price_mu=None):
        self.buy_price_mu = buy_price_mu
        self.sell_price_mu = sell_price_mu
# class info, which contains the properties: branch_info, bus_reference, voltage_limits, pu_values and cables_characteristics.
class Info:
    def __init__(self, branch_info=None, bus_reference=None, voltage_limits=None, pu_values=None, cable_characteristics=None):
        self.branch_info = branch_info
        self.bus_reference = bus_reference
        self.voltage_limits = voltage_limits
        self.pu_values = pu_values
        self.cables_characteristics = cable_characteristics
##############################################################################
############################## Data: Full Excell #############################
##############################################################################
# class Data constains the properties: simulation_periods, periods_duration_min, objective_functions_list, a dict called network_information, a list of objects of the class Vehicle, a list of objects of the class Load, a list of objects of the class Generator, a list of objects of the class Storage, a list of objects of the class Charging_Station, a list of objects of the class Peer.
class Network:
    def __init__(
        self, simulation_periods=None, periods_duration_min=None, objective_functions_list=None,
        info=None, vehicle_list=None, load_list=None, generator_list=None,
        storage_list=None, charging_station_list=None, peer_list=None):
        self.simulation_periods = simulation_periods
        self.periods_duration_min = periods_duration_min
        self.objective_functions_list = objective_functions_list
        self.info = info
        self.vehicle_list = vehicle_list
        self.load_list = load_list
        self.generator_list = generator_list
        self.storage_list = storage_list
        self.charging_station_list = charging_station_list
        self.peer_list = peer_list
    # Methods.
    def create_network_from_xlsx(self, xlsx_file_path):
        print(os.getcwd() + '\\' + xlsx_file_path)
        # create one dataframe for the General_Information sheet information in the excel file
        raw_general_information_sheet = pd.read_excel(os.getcwd() + '\\' + xlsx_file_path, sheet_name='General_Information')
        # create one dataframe for the Peers_info sheet information in the excel file
        raw_peers_sheet = pd.read_excel(os.getcwd() + '\\' + xlsx_file_path, sheet_name='Peers_Info')
        # create one dataframe for the Load sheet information in the excel file
        raw_load_sheet = pd.read_excel(os.getcwd() + '\\' + xlsx_file_path, sheet_name='Load')
        # create one dataframe for the Generator sheet information in the excel file
        raw_generator_sheet = pd.read_excel(os.getcwd() + '\\' + xlsx_file_path, sheet_name='Generator')
        # create one dataframe for the Storage sheet information in the excel file
        raw_storage_sheet = pd.read_excel(os.getcwd() + '\\' + xlsx_file_path, sheet_name='Storage')
        # create one dataframe for the Vehicle sheet information in the excel file
        raw_vehicle_sheet = pd.read_excel(os.getcwd() + '\\' + xlsx_file_path, sheet_name='Vehicle')
        # create one dataframe for the CStation sheet information in the excel file
        raw_charging_station_sheet = pd.read_excel(os.getcwd() + '\\' + xlsx_file_path, sheet_name='CStation')
        # create one dataframe for the Network_Info sheet information in the excel file
        raw_network_info_sheet = pd.read_excel(os.getcwd() + '\\' + xlsx_file_path, sheet_name='Network_Info')
        # create one dataframe for the General_Information sheet information in the excel file
        raw_general_information_sheet = pd.read_excel(os.getcwd() + '\\' + xlsx_file_path, sheet_name='General_Information')
        # Extract useful info from the raw dataframes.
        extractor = Extractor()
        # Create network object
        self.simulation_periods = extractor.create_simulation_periods(raw_general_information_sheet)
        self.periods_duration_min = extractor.create_periods_duration_min(raw_general_information_sheet)
        self.objective_functions_list = extractor.create_objective_functions_list(raw_general_information_sheet)
        self.info = extractor.create_network_info(raw_network_info_sheet)
        self.vehicle_list = extractor.create_vehicles_list(raw_vehicle_sheet)
        self.load_list = extractor.create_loads_list(raw_load_sheet)
        self.generator_list = extractor.create_generators_list(raw_generator_sheet)
        self.storage_list = extractor.create_storages_list(raw_storage_sheet)
        self.charging_station_list = extractor.create_charging_stations_list(raw_charging_station_sheet)
        self.peer_list = extractor.create_peers_list(raw_peers_sheet)
        del self.generator_list[-1]
    def create_pandapower_model(self):
        """Method that created a pandapower network model.
        """
        net = pp.create_empty_network()
        # Create the busses.
        busses = pd.concat([self.info.branch_info.bus_out, self.info.branch_info.bus_in], axis=0).unique()
        for bus in busses:
            name = 'bus_' + str(bus)
            pp.create_bus(net, vn_kv=20.0, index=bus, name=name)
        # External grid.
        pp.create_bus(net, vn_kv=110, index=0, name='ext_grid')
        pp.create_ext_grid(net, bus=0, vm_pu=1.0)
        # Create transformer.
        pp.create_transformer(net, hv_bus=0, lv_bus=1, std_type='40 MVA 110/20 kV') # TODO see!
        # Create tge loads from network.load_list.
        for i, load in enumerate(self.load_list):
            name = 'load_' + str(load.id)
            bus = load.internal_bus_location
            p_mw = load.power_contracted_kw / 1000
            q_mvar  = p_mw * load.tg_phi
            pp.create_load(net, bus=bus, p_mw=p_mw, q_mvar=q_mvar, name=name)
        # Create generators from network.generator_list.
        for i, generator in enumerate(self.generator_list):
            name = 'gen_' + str(generator.id)
            bus = generator.internal_bus_location
            p_mw = generator.p_max_kw / 1000
            q_mvar = generator.q_max_kvar / 1000
            pp.create_sgen(net, bus=bus, p_mw=p_mw, q_mvar=q_mvar, name=name)    
        # Create the storage from network.storage_list.
        for i, storage in enumerate(self.storage_list):
            name = 'stor_' + str(storage.id)
            bus = storage.internal_bus_location
            p_mw = storage.p_charge_max_kw / 1000
            max_e_mwh = storage.energy_capacity_kvah / 1000
            pp.create_storage(net, bus=bus, p_mw=p_mw, max_e_mwh=max_e_mwh, name=name)
        # Create lines from network.info.branch_info.
        cables_characteristics = self.info.cables_characteristics.set_index('name')
        for row in self.info.branch_info.itertuples():
            from_bus = row.bus_out
            to_bus = row.bus_in
            length_km = row.distance_km
            c_nf_per_km = row.c
            name = 'line_' + str(row.bus_out) + '_to_' + str(row.bus_in) 
            r_ohm_per_km = cables_characteristics.loc[row.cable_type].r
            x_ohm_per_km = cables_characteristics.loc[row.cable_type].x
            pp.create_line_from_parameters(
                net, from_bus=from_bus, to_bus=to_bus, 
                length_km=length_km, r_ohm_per_km=r_ohm_per_km,
                x_ohm_per_km=x_ohm_per_km, c_nf_per_km=c_nf_per_km, max_i_ka=1, name=name
                )
        self.net_model = net
    def get_pandapower_model(self):
        """Method that returns the model of the network.
        Returns:
            pandapower.net: Model of the network.
        """
        return self.net_model
    def plot_network(self, power_flow=False):
        """Function that plots the network. 
        Args:
            power_flow (bool, optional): Perfoms a power flow on the network. Defaults to True.
        """
        fig = simple_plotly(self.net_model, respect_switches=True, figsize=2)  
        if power_flow:
            fig = pf_res_plotly(self.net_model, figsize=2)
    def add_generation_profiles(self, generation_profiles_folder_path=None):
        """Function that adds the generation profiles to the network.
        Args:
            generation_profiles_path (str): Path to the generation profiles.
        """
        folder_path = generation_profiles_folder_path
        pv_generation_profile = pd.read_csv(folder_path + '\\' + 'pv_data_processed.csv')
        ut.convert_to_timestamped_data(pv_generation_profile)
        wind_generation_profile = pd.read_csv(folder_path + '\\' + 'wind_data_processed.csv')
        ut.convert_to_timestamped_data(wind_generation_profile)
        # Multiply the normalized value by the installed capacity of the generator.
        for generator in self.generator_list:
            if generator.type_of_generator == 'PV':
                new_profile = pv_generation_profile['normalized_value'] * generator.p_max_kw 
                generator.p_real_kw = new_profile * np.random.uniform(0.85, 1.15, new_profile.shape)
            elif generator.type_of_generator == 'Wind':
                new_profile = wind_generation_profile['normalized_value'] * generator.p_max_kw 
                generator.p_real_kw = new_profile * np.random.uniform(0.85, 1.15, new_profile.shape)
            elif generator.type_of_generator == 'CHP':
                chp_profile = pd.DataFrame(index = pv_generation_profile.index, columns = ['value'])
                for i, time in enumerate(chp_profile.index):
                    if time.hour < 9 or time.hour > 17:
                        chp_profile.iloc[i] = 0
                    else:
                        chp_profile.iloc[i] = generator.p_max_kw        
                generator.p_real_kw = chp_profile
            elif generator.type_of_generator == 'External Suplier':
                pass 
            else: 
                # raise error with the generator type is not supported.
                raise NameError("The generator type is not supported.")
    def add_load_profiles(self, load_profiles_folder_path=None):
        """Function that adds the load profiles to the network.
        Args:
            load_profiles_path (str): Path to the load profiles.
        """
        folder_path = load_profiles_folder_path
        consumption_file_list = os.listdir(folder_path)
        for (file, load) in zip(consumption_file_list, self.load_list):
            # Read the file.
            new_profile = pd.read_csv(folder_path + '\\' + file)
            ut.convert_to_timestamped_data(new_profile)
            # Add profile to element of te grid.
            if new_profile['normalized_P'].isna().sum() != 0:
                print('Stop')
            load.p_real_kw = new_profile['normalized_P'] * load.power_contracted_kw 
            load.q_real_kvar = new_profile['normalized_Q'] * load.power_contracted_kw * load.tg_phi
        print('Profile done.')

In [ ]:
#%%
import pandas as pd
import os
#from thesis_package import elements
#import re
##############################################################################
#################### Extractor Functions: Peers ##############################
##############################################################################
# Function that takes as input raw_network_info_sheet and outputs a list of Peer class objects initialized with the data from raw_network_info_sheet.
class Extractor: 
    def create_peers_list(self, df):
        """Function that creates the list of peers from the raw_network_info_sheet.
        Args:
            df (pandas.dataframe): Compilation of dataframes from the raw_network_info_sheet.

        Returns:
            list: Returns a list of elements.Peer objects.
        """
        # create n empty list to store Peer objects
        peers = []
        peers_info = self.organize_raw_data(df)
        for i in range(len(peers_info['peer_id'])):
            new_peer = elements.Peer(
                id=peers_info['peer_id'].loc[i],
                type_of_peer=peers_info['type_of_peer'].loc[i].values[0],
                type_of_contract=peers_info['type_of_contract'].loc[i].values[0]
                )
            new_peer.set_profile(
                buy_price_mu=peers_info['buy_price_mu'].loc[i],
                sell_price_mu=peers_info['sell_price_mu'].loc[i]
                )
            peers.append(new_peer)
        return peers
    def create_vehicles_list(self, df):
        """Function that creates the list of vehicles from the raw_vehicle_sheet.

        Args:
            df (pandas.dataframe): Compilation of dataframes from the raw_vehicle_sheet.

        Returns:
            list: Returns a list of elements.Vehicle objects.
        """
        # Create empty list to store Vehicle objects.
        vehicles = []
        vehicles_info = self.organize_raw_data(df)
        for i in range(len(vehicles_info['electric_vehicle_id'])):
            new_vehicle = elements.Vehicle(
                id=vehicles_info['electric_vehicle_id'].loc[i],
                owner=vehicles_info['owner'].loc[i].values[0],
                manager=vehicles_info['manager'].loc[i].values[0],
                type_of_vehicle=vehicles_info['type_of_vehicle'].loc[i].values[0],
                type_of_contract=vehicles_info['type_of_contract'].loc[i].values[0],
                energy_capacity_max_kwh=vehicles_info['e_capacity_max_kwh'].loc[i].values[0],
                p_charge_max_kw=vehicles_info['p_charge_max_kw'].loc[i].values[0],
                p_discharge_max_kw=vehicles_info['p_discharge_max_kw'].loc[i].values[0],
                charge_efficiency_percent=vehicles_info['charge_efficiency'].loc[i].values[0],
                discharge_efficiency_percent=vehicles_info['discharge_efficiency'].loc[i].values[0],
                initial_state_SOC_percent=vehicles_info['initial_state_soc'].loc[i].values[0],
                minimun_technical_SOC_percent=vehicles_info['minimun_technical_soc'].loc[i].values[0]
                )
            new_vehicle.set_profile(
                arrive_time_period = vehicles_info['arrive_time_period'].iloc[i],
                departure_time_period = vehicles_info['departure_time_period'].iloc[i],
                place = vehicles_info['place'].iloc[i],
                used_soc_percent_arriving = vehicles_info['used_soc_arriving'].iloc[i],
                soc_percent_arriving = vehicles_info['soc_arriving'].iloc[i],
                soc_required_percent_exit = vehicles_info['soc_required_exit'].iloc[i],
                p_charge_max_constracted_kw = vehicles_info['pcharge_max_contracted_kw'].iloc[i],
                p_discharge_max_constracted_kw = vehicles_info['pdcharge_max_contracted_kw'].iloc[i],
                charge_price = vehicles_info['charge_price'].iloc[i],
                discharge_price = vehicles_info['disharge_price'].iloc[i]
                )
            vehicles.append(new_vehicle)
        return vehicles 
    def create_charging_stations_list(self, df):
        """Function that creates the list of charging stations from the raw_charging_station_sheet.

        Args:
            df (pandas.dataframe): Compilation of dataframes from the raw_charging_station_sheet.

        Returns:
            list: Returns a list of elements.ChargingStation objects.
        """
        # Create empty list to store ChargingStation objects.
        charging_stations = []
        charging_stations_info = self.organize_raw_data(df)
        for i in range(len(charging_stations_info['charging_station_id'])):
            new_charging_station = elements.Charging_Station(
                id=charging_stations_info['charging_station_id'].loc[i],
                internal_bus_location=charging_stations_info['internal_bus_location'].loc[i].values[0],
                manager=charging_stations_info['manager'].loc[i].values[0],
                owner=charging_stations_info['owner'].loc[i].values[0],
                type_of_contract=charging_stations_info['type_of_contract'].loc[i].values[0],
                p_charge_max_kw=charging_stations_info['p_charge_max_kw'].loc[i].values[0],
                p_discharge_max_kw=charging_stations_info['p_discharge_max_kw'].loc[i].values[0],
                charge_efficiency_percent=charging_stations_info['charge_efficiency'].loc[i].values[0],
                discharge_efficiency_percent=charging_stations_info['discharge_efficiency'].loc[i].values[0],
                energy_capacity_max_kwh=charging_stations_info['e_capacity_max_kwh'].loc[i].values[0],
                place_start=charging_stations_info['place_start'].loc[i].values[0],
                place_end=charging_stations_info['place_end'].loc[i].values[0]
                )
            new_charging_station.set_profile(
                power_charge_limit_kw=charging_stations_info['p_charge_limit_kw'].loc[i],
                p_discharge_limit_kw=charging_stations_info['p_charge_limit_kw'].loc[i]
                )
            charging_stations.append(new_charging_station)
        return charging_stations
    def create_generators_list(self, df):
        """Function that creates the list of generators from the raw_generator_sheet.

        Args:
            df (pandas.dataframe): Compilation of dataframes from the raw_generator_sheet.

        Returns:
            list: Returns a list of elements.Generator objects.
        """
        # Create empty list to store Generator objects.
        generators = []
        generators_info = self.organize_raw_data(df)
        for i in range(len(generators_info['generator_id'])):
            new_generator = elements.Generator(
                id=generators_info['generator_id'].loc[i],
                internal_bus_location=generators_info['internal_bus_location'].loc[i].values[0],
                manager=generators_info['manager'].loc[i].values[0],
                owner=generators_info['owner'].loc[i].values[0],
                type_of_contract=generators_info['type_of_contract'].loc[i].values[0],
                type_of_generator=generators_info['type_of_generator'].loc[i].values[0],
                p_max_kw=generators_info['p_max_kw'].loc[i].values[0],
                p_min_kw=generators_info['p_min_kw'].loc[i].values[0],
                q_max_kvar=generators_info['q_max_kvar'].loc[i].values[0],
                q_min_kvar=generators_info['q_min_kvar'].loc[i].values[0]
            )
            new_generator.set_profile(
                power_real_kw=generators_info['p_forecast_kw'].loc[i],
                cost_parameter_a_mu = generators_info['cost_parameter_a_mu'].loc[i],
                cost_parameter_b_mu = generators_info['cost_parameter_b_mu'].loc[i],
                cost_parameter_c_mu = generators_info['cost_parameter_c_mu'].loc[i],
                cost_nde_mu = generators_info['cost_nde_mu'].loc[i],
                ghg_cof_a_mu = generators_info['ghg_cof_a_mu'].loc[i],
                ghg_cof_b_mu = generators_info['ghg_cof_b_mu'].loc[i],
                ghg_cof_c_mu = generators_info['ghg_cof_c_mu'].loc[i]
            )
            generators.append(new_generator)
        return generators
    def create_storages_list(self, df):
        """Function that creates the list of storages from the raw_storage_sheet.

        Args:
            df (pandas.dataframe): Compilation of dataframes from the raw_storage_sheet.

        Returns:
            list: Returns a list of elements.Storage objects.
        """
        # Create empty list to store Storage objects.
        storages = []
        storages_info = self.organize_raw_data(df)
        for i in range(len(storages_info['storage_id'])):
            new_storage = elements.Storage(
                id=storages_info['storage_id'].loc[i],
                internal_bus_location=storages_info['internal_bus_location'].loc[i].values[0],
                manager=storages_info['manager'].loc[i].values[0],
                owner=storages_info['owner'].loc[i].values[0],
                type_of_contract=storages_info['type_of_contract'].loc[i].values[0],
                battery_type=storages_info['battery_type'].loc[i].values[0],
                energy_capacity_kvah=storages_info['energy_capacity_kvah'].loc[i].values[0],
                energy_min_percent=storages_info['energy_min'].loc[i].values[0],
                charge_efficiency_percent=storages_info['charge_efficiency'].loc[i].values[0],
                discharge_efficiency_percent=storages_info['discharge_efficiency'].loc[i].values[0],
                initial_state_percent=storages_info['initial_state'].loc[i].values[0],
                p_charge_max_kw=storages_info['p_charge_max_kw'].loc[i].values[0],
                p_discharge_max_kw=storages_info['p_discharge_max_kw'].loc[i].values[0]
                )
            new_storage.set_profile(
                power_charge_limit_kw=storages_info['p_charge_limit_kw'].loc[i],
                power_discharge_limit_kw=storages_info['p_discharge_limit_kw'].loc[i],
                charge_price_mu=storages_info['charge_price_mu'].loc[i],
                discharge_price_mu=storages_info['discharge_price_mu'].loc[i]
            )
            storages.append(new_storage)
        return storages       
    def create_loads_list(self, df):
        """Function that creates the list of loads from the raw_load_sheet.

        Args:
            df (pandas.dataframe): Compilation of dataframes from the raw_load_sheet.

        Returns:
            list: Returns a list of elements.Load objects.
        """
        # Create empty list to store Load objects.
        loads = []
        loads_info = self.organize_raw_data(df)
        for i in range(len(loads_info['load_id'])):
            new_load = elements.Load(
                id=loads_info['load_id'].loc[i],
                internal_bus_location=loads_info['internal_bus_location'].loc[i].values[0],
                manager=loads_info['manager_id'].loc[i].values[0],
                owner=loads_info['owner_id'].loc[i].values[0],
                type_of_contract=loads_info['type_of_contract'].loc[i].values[0],
                charge_type=loads_info['charge_type'].loc[i].values[0],
                power_contracted_kw=loads_info['p_contracted_kw'].loc[i].values[0],
                tg_phi=loads_info['tg_phi'].loc[i].values[0]
            )            
            new_load.set_profile(
                p_real_kw=loads_info['p_forecast_kw'].loc[i],
                q_real_kvar=loads_info['q_forecast_kvar'].loc[i],
                p_reduce_kw=loads_info['p_reduce_kw'].loc[i],
                p_cut_kw=loads_info['p_cut_kw'].loc[i],
                p_move_kw=loads_info['p_move_kw'].loc[i],
                p_in_move_kw=loads_info['p_in_move_kw'].loc[i],
                cost_reduce_mu=loads_info['cost_reduce_mu'].loc[i],
                cost_cut_mu=loads_info['cost_cut_mu'].loc[i],
                cost_mov_mu=loads_info['cost_mov_mu'].loc[i],
                cost_ens_mu=loads_info['cost_ens_mu'].loc[i]
            )
            loads.append(new_load)
        return loads
    def create_network_info(self, df):
        network_info = {}
        # Bus reference
        _dict =  { self.format_string(df.iloc[0,i]) : [df.iloc[1,i]]
         for i in range(3)}
        df_title = self.format_string(df.columns[0])
        network_info[df_title] = pd.DataFrame(_dict)
        # Voltage limits
        df_title = self.format_string(df.columns[4])
        _dict1 = {self.format_string(df.iloc[0,i]) : [df.iloc[1,i]] for i in range(4,6)} 
        _dict2 = {self.format_string(df.iloc[2,i]) : [df.iloc[3,i]] for i in range(4,6)} 
        _dict = {**_dict1, **_dict2}
        network_info[df_title] = pd.DataFrame(_dict)
        # PU values
        df_title = self.format_string(df.columns[7])
        columns = df.iloc[:,7].dropna().unique().tolist()
        columns[-2:] = []
        _dict = {self.format_string(column): [ df['Unnamed: 8'][i] ] for i, column in enumerate(columns)}
        network_info[df_title] = pd.DataFrame(_dict)
        # Branch info
        columns = df.iloc[10,:].dropna().unique().tolist()
        columns[columns.index('ohm/km'):] = []
        df_title = self.format_string(df.iloc[9,0])
        _dict = { self.format_string(column): df.iloc[11:,i].dropna().tolist() for i, column in enumerate(columns)}
        network_info[df_title] = pd.DataFrame(_dict)
        # Cable characteristics.
        df_title = self.format_string(df['Unnamed: 10'][9])
        columns = df.iloc[11,:].dropna().unique().tolist()
        columns[:8] = []
        columns = [self.format_string(col) for col in columns]
        columns = ['name'] + columns
        _dict = {column: df.iloc[12:,10+i].dropna().tolist() for i, column in enumerate(columns)}
        network_info[df_title] = pd.DataFrame(_dict)
        # Initialize the Info object.
        new_info = elements.Info(
            branch_info=network_info['branch_info'],
            bus_reference=network_info['bus_reference'],
            voltage_limits=network_info['voltage_limits'],
            pu_values=network_info['pu_values'],
            cable_characteristics=network_info['cables_characteristics']
            )
        return new_info
    def create_simulation_periods(self, df):
        return df[df['Unnamed: 2'] == 'Simulation Periods']['Unnamed: 3'].values[0]
    def create_periods_duration_min(self, df):
        return df[df['Unnamed: 2'] == 'Periods Duration (min)']['Unnamed: 3'].values[0]
    def create_objective_functions_list(self, df):
        columns = df['Unnamed: 6'].dropna().tolist()
        return pd.DataFrame({column : [i+1] for i, column in enumerate(columns)})
    ############################# Aux Functions ###################################
    # Extract useful info from the raw_peer_info_sheet into a new dataframe.
    def organize_raw_data(self, df):
        """Function that receives a raw information dataframe and returns a new dataframe with the useful information.

        Args:
            df (pandas.dataframe): A dataframe with the raw information extracted from a xlsx file.

        Returns:
            dict: A dictionary with the useful information extracted from the raw_peer_info_sheet.
        """
        # Get static info names from raw_generator_sheet.
        # Get all values of the column 'Unnamed: 2', remove empty values, removed duplicate values, and convert to list.
        static_info_names = df['Unnamed: 2'].dropna().unique().tolist()
        static_info_names[0] = df.columns[0]
        # Get the index of the column 'Total Time (h)' in the list of column names
        # Get all values of the column 'Total Time (h)', remove empty values, removed duplicate values, and convert to list.
        if static_info_names[0] == 'Electric Vehicle ID': 
            dynamic_info_names = df['Unnamed: 5'].dropna().unique().tolist()
            dynamic_info_names[:2] = []
        else:    
            dynamic_info_names = df['Total Time (h)'].dropna().unique().tolist()
            dynamic_info_names[:2] = []

        # Static Data
        info_dict = {}
        # iterate through the static_info_names and for each name create a new dataframe with info extracted from the raw information sheets.
        for i, static_info_name in enumerate(static_info_names):
            if i == 0: # The the ID
                # Create the new dataframe with the extracted information.
                # Substitute space by underscore in the name of the column.
                _df_name = static_info_name.lower().replace(' ', '_') 
                info_dict[_df_name] = pd.DataFrame()
                info_dict[_df_name] = df[static_info_names[0]].filter(regex='^\d+$').dropna().drop(1).reset_index(drop=True).rename(re.sub('[^0-9a-zA-Z]+', '_', static_info_name).rstrip('_'))
            else: # The rest of the static info
                # All characters of _df_name are converted to lower case, all charaters that are not numbers or letters are removed, all white spaces are replaced by underscores, and if the last character is an underscore, it is removed.
                _df_name = re.sub('[^0-9a-zA-Z]+', '_', static_info_name).rstrip('_').lower()
                info_dict[_df_name] = pd.DataFrame()
                info_dict[_df_name] = df[df['Unnamed: 2'].str.contains(static_info_name.split('(')[0]).fillna(False)]['Unnamed: 3'].reset_index().drop(['index'], axis=1).rename(columns={'Unnamed: 3': _df_name})   
        # Dynamic Data
        if static_info_names[0] == 'Electric Vehicle ID':
            total_time_index = df.columns.get_loc('Unnamed: 5')
        else:
            total_time_index = df.columns.get_loc('Total Time (h)')
        # Get the indexes of all the columns after 'Total Time (h)'.
        # This will be used to get the values of the columns that we want to keep
        # and store themss in a list.
        columns_to_keep_profiles = df.columns[total_time_index+1:].tolist() 
        for dynamic_info_name in dynamic_info_names:
            _dynamic_info_name = dynamic_info_name.replace('.', '')
            _df_name = re.sub('[^0-9a-zA-Z]+', '_', _dynamic_info_name).rstrip('_').lower()
            info_dict[_df_name] = pd.DataFrame()
            info_dict[_df_name] = df[df.iloc[:,5].apply(lambda x: x  == dynamic_info_name)].iloc[:][columns_to_keep_profiles].reset_index(drop=True)
        return info_dict
    def format_string(self, string):
        return re.sub('[^0-9a-zA-Z]+', '_', string).rstrip('_').lower()
# Cable Carachteristics ask questions
# Note: When finish last sheet, ask the prof about the last sheets